<a href="https://colab.research.google.com/github/JoelTomyGeorge/BizRecProject/blob/main/Merge_CSVs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
!pip install findspark

In [ ]:

!pip install pyarrow


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import *
import time

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# Initialize SparkSession
spark = SparkSession.builder.appName("CSVReader").getOrCreate()

In [ ]:
# prompt: read /content/drive/MyDrive/ProcessedCSV/business.csv, make sure everythings in memory, also print first 2 rec

df_business = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/business.csv", header=True, inferSchema=True)
df_business = df_business.withColumnRenamed("stars","stars_bus" ).withColumnRenamed("name","bus_name" ).withColumnRenamed("review_count","bus_review_count").cache()
# df_business.show(2)


In [ ]:
df_user = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/user.csv", header=True, inferSchema=True)
df_user = df_user.withColumnRenamed("cool","cool_usr" ).withColumnRenamed("funny","funny_usr" ).withColumnRenamed("useful","useful_usr").withColumnRenamed("name","usr_name")
df_user = df_user.drop("date", "time").cache()
# df_user.show(2)

In [ ]:
df_review = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/review.csv", header=True, inferSchema=True)
df_review = df_review.withColumnRenamed("cool","cool_rev" ).withColumnRenamed("funny","funny_rev" ).withColumnRenamed("useful","useful_rev" ).withColumnRenamed("text","rev_text" ).withColumnRenamed("stars","stars_rev")
df_review = df_review.drop("date", "time").cache()
# df_review.show(2)

In [ ]:
df_tip = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/tip.csv", header=True, inferSchema=True)
df_tip = df_tip.withColumnRenamed("text","tip_text" )
df_tip = df_tip.drop("date", "time").cache()
# df_tip.show(2)

In [ ]:
df_checkin = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/checkin.csv", header=True, inferSchema=True)
df_checkin = df_checkin.drop("date", "time").cache()
# df_checkin.show(2)

In [ ]:
# prompt: I need to merge all this data frame to a simgle one, business has business_id, user has user_id, review has review_id, user_id, and business_id,  tip has user_id and business_id , checkin has business_id

# Merge review with business and user
df_merged_rev_bus = df_review.join(df_business, on='business_id', how='inner')
df_business.unpersist(blocking=True)
df_review.unpersist(blocking=True)

#writing as a parquet file
df_merged_rev_bus.write.mode("overwrite").parquet("/content/drive/MyDrive/parquetfiles/merged_rev_bus.parquet")

#reading the parquet saved before
df_merged_rev_bus = spark.read.parquet("/content/drive/MyDrive/parquetfiles/merged_rev_bus.parquet")
#joining the user to this
df_merged_rev_bus_usr = df_merged_rev_bus.join(df_user, on='user_id', how='inner')
df_user.unpersist(blocking=True)
# writing the new parquet
df_merged_rev_bus_usr.write.mode("overwrite").parquet("/content/drive/MyDrive/parquetfiles/merged_rev_bus_usr.parquet")

# reading the parquet
df_merged_rev_bus_usr = spark.read.parquet("/content/drive/MyDrive/parquetfiles/merged_rev_bus_usr.parquet")
# Merge tip with the merged dataframe
df_merged_rev_bus_usr_tip = df_merged_rev_bus_usr.join(df_tip, on=['user_id', 'business_id'], how='left')
df_tip.unpersist(blocking=True)
# writing the new parquet
df_merged_rev_bus_usr_tip.write.mode("overwrite").parquet("/content/drive/MyDrive/parquetfiles/merged_rev_bus_usr_tip.parquet")

#reading the parquet
df_merged_rev_bus_usr_tip = spark.read.parquet("/content/drive/MyDrive/parquetfiles/merged_rev_bus_usr_tip.parquet")
# Merge checkin with the merged dataframe
df_merged_rev_bus_usr_tip_checkin = df_merged_rev_bus_usr_tip.join(df_checkin, on='business_id', how='left').cache()
df_checkin.unpersist(blocking=True)
# writing the new parquet
df_merged_rev_bus_usr_tip_checkin.write.mode("overwrite").parquet("/content/drive/MyDrive/parquetfiles/finalmerged.parquet")

# df_merged.show(2)


In [ ]:
# prompt: write merged as a parquet file
# Initialize SparkSession

# df_merged.write.mode("overwrite").parquet("/content/drive/MyDrive/mergedcsv/merged_data.parquet")


In [ ]:
# prompt: read and display 2 rows of the parquet

# df_parquet = spark.read.parquet("/content/drive/MyDrive/ProcessedCSV/merged_data_parquet.parquet")
# df_parquet.show(2)


In [ ]:
# prompt: write this merged df in /content/drive/MyDrive/ProcessedCSV named merged.csv

df_merged.write.mode("overwrite").csv("/content/drive/MyDrive/ProcessedCSV/merged.csv", header=True)


In [ ]:
# prompt: read and display "/content/drive/MyDrive/ProcessedCSV/merged.csv"

df_merged = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/merged.csv", header=True, inferSchema=True)
df_merged.show()


+--------------------+--------------------+--------------------+---------+----------+---------+--------+--------------------+--------------------+--------------------+--------------+-----+-----------+-------------+--------------+---------+----------------+-------+----------------------------+--------------------+-----+-------------------------------------+------------+-------------+---------------+--------------+------------+--------------+----------------------+---------------------------------+--------------------+-----------------------------+------------------------------+-----------------+---------------+--------------------------+-------------------------------+--------------------+-------------------------+----------------+----------------------------------+----------------------+------------+------------------+----------------------+----------------------------+--------------------+----------------------------------+-----------------------------------+--------------------+-----